In [1]:
# Dependencies 
import pandas as pd
import numpy as np
import os

import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, Text

In [2]:
# Create a Database Connection
engine = create_engine('sqlite:///hawaii.sqlite')

In [3]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [4]:
# Define the Measurement class
Base = declarative_base()

class Measurement(Base):
    __tablename__ = "measurement"
    
    index = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(Text)
    prcp = Column(Float) 
    tobs = Column(Integer) 

    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [5]:
# Define the Station class

class Station(Base):
    __tablename__ = "station"
    
    station = Column(String, primary_key=True)
    name = Column(String)
    latitude = Column(Float) 
    longitude = Column(Float) 
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [6]:
# Use `create_all` to create the demographics table in the database

Base.metadata.create_all(engine)

In [7]:
# Read CSV files into a pandas DataFrame
hawaii_measurements_df = pd.read_csv("data/clean_hawaii_measurements.csv")
hawaii_stations_df = pd.read_csv("data/clean_hawaii_stations.csv")

In [8]:
# Use Orient='records' to create a list of data to write
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
data = hawaii_measurements_df.to_dict(orient="records")
print(data[:1])

[{'station': 'USC00519397', 'date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}]


In [9]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [10]:
# Save the reference to the `measurement` table as a variable called `table` 
# Bring the table into python using SQLAlchemy
measurement_table = sqlalchemy.Table('measurement', metadata, autoload=True)

In [11]:
# Use `table.delete()` to remove any existing data.

conn.execute(measurement_table.delete())

In [12]:
# Use `table.insert()` to insert the data into the table
conn.execute(measurement_table.insert(), data)

In [13]:
# Use Orient='records' to create a list of data to write
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
data = hawaii_stations_df.to_dict(orient="records")
print(data[:1])

[{'station': 'USC00519397', 'name': 'WAIKIKI 717.2, HI US', 'latitude': 21.2716, 'longitude': -157.8168, 'elevation': 3.0}]


In [14]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [15]:
# Save the reference to the `station` table as a variable called `table` 
# Bring the table into python using SQLAlchemy
table = sqlalchemy.Table('station', metadata, autoload=True)

In [16]:
# Use `table.delete()` to remove any existing data.

conn.execute(table.delete())

In [17]:
# Use `table.insert()` to insert the data into the table
conn.execute(table.insert(), data)

In [18]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from station limit 10").fetchall()

[('USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 ('USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 ('USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 ('USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 ('USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 ('USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999998, 32.9),
 ('USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 ('USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]

In [19]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from measurement limit 10").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-07', 0.06, 70),
 (6, 'USC00519397', '2010-01-08', 0.0, 64),
 (7, 'USC00519397', '2010-01-09', 0.0, 68),
 (8, 'USC00519397', '2010-01-10', 0.0, 73),
 (9, 'USC00519397', '2010-01-11', 0.01, 64),
 (10, 'USC00519397', '2010-01-12', 0.0, 61)]